<h1>Algoritmo Backpropagation</h1>
<h4>Integrantes: Javier Eduardo Gutierrez Serrano - Wilder Ofrey Bello Herrera</h4>

In [1]:
# Probando un poco de lo que he entendido y encontrado. Con aplicación de objetos.
import numpy as np 
import pandas as pd 
from numpy import random 
import matplotlib.pyplot as plt 

In [2]:
def tanh(x):
    return np.tanh(x)
def dtanh(x):
    return 1.0 - np.tanh(x)**2
def entrenar(m, x_1, x_n):
    return m[:,x_1:x_n+1]
def validar(m, x_ij, x_jn):
    return m[:,x_ij:x_jn+1]

In [3]:
class PerceptronMulti():
    def __init__(self,xi,d,w_1,w_2,us,uoc,precision,epocas,fac_ap,n_ocultas,n_entradas,n_salida):
        self.xi = np.transpose(xi)
        self.d = d
        self.w1 = w_1
        self.w2 = w_2
        self.us = us
        self.uoc = uoc
        self.precision = precision
        self.epocas = epocas
        self.fac_ap = fac_ap
        self.n_entradas = n_entradas
        self.n_ocultas = n_ocultas
        self.n_salida = n_salida
        
        self.di = 0 
        self.error_red = 1
        self.Ew = 0 
        self.Error_prev = 0 
        self.Errores = []
        self.Error_actual = np.zeros((len(d)))
        self.Entradas = np.zeros((1,n_entradas))
        self.un = np.zeros((n_ocultas,1)) 
        self.gu = np.zeros((n_ocultas,1))
        self.Y = 0.0 
        self.y = 0.0 
        self.epochs = 0
        
        self.error_real = 0
        self.ds = 0.0 
        self.docu = np.zeros((n_ocultas,1))
        
    def Operation(self):
        respuesta = np.zeros((len(self.d),1))
        for p in range(len(self.d)):
            self.Entradas = self.xi[:,p]
            self.Propagar()
            respuesta[p,:] = self.y
        return respuesta.tolist()
    
    def Learn(self):
        Errores = []
        while(np.abs(self.error_red) > self.precision):
            self.Error_prev = self.Ew
            for i in range(len(d)):
                self.Entradas = self.xi[:,i] 
                self.di = self.d[i]
                self.Propagar()
                self.Backpropagation()
                self.Propagar()
                self.Error_actual[i] = (0.5)*((self.di - self.y)**2)
            self.Error()
            Errores.append(self.error_red)
            self.epochs +=1
            if self.epochs > self.epocas:
                break
        return self.epochs,self.w1,self.w2,self.us,self.uoc,Errores
                
    
    def Propagar(self):
        for a in range(self.n_ocultas):
            self.un[a,:] = np.dot(self.w1[a,:], self.Entradas) + self.uoc[a,:]
        
        for o in range(self.n_ocultas):
            self.gu[o,:] = tanh(self.un[o,:])
        
        self.Y = (np.dot(self.w2,self.gu) + self.us)
        self.y = tanh(self.Y)
    
    def Backpropagation(self):
        self.error_real = (self.di - self.y)
        self.ds = (dtanh(self.Y) * self.error_real)
        self.w2 = self.w2 + (np.transpose(self.gu) * self.fac_ap * self.ds)
        self.us = self.us + (self.fac_ap * self.ds)
        self.docu = dtanh(self.un) * np.transpose(self.w2) * self.ds
        for j in range(self.n_ocultas):
            self.w1[j,:] = self.w1[j,:] + ((self.docu[j,:]) * self.Entradas * self.fac_ap)
        
        for g in range(self.n_ocultas):
            self.uoc[g,:] = self.uoc[g,:] + (self.fac_ap * self.docu[g,:])
        
    def Error(self):
        self.Ew = ((1/len(d)) * (sum(self.Error_actual)))
        self.error_red = (self.Ew - self.Error_prev)

if "__main__" == __name__:
    archivo = pd.ExcelFile('Data.xlsx')
    data = archivo.parse('Hoja1')
    matriz_data = np.array(data)
    x_init = 0
    x_n = 1
    x_j_init = 3
    x_j_n = 4
    x_i = (entrenar(matriz_data, x_init, x_n))
    d = matriz_data[:, x_n+1]
    x_j = (validar(matriz_data, x_j_init, x_j_n))
    f,c = x_i.shape
    f_ac_ap = 0.2
    prec = 0.00000001
    epoch = 10000
    epochs = 0 
    num_inputs = c
    shadow_caps = 2
    neuronas = 6
    output = 1
    umbral = 1.0
    umbral_ocultas = np.ones((neuronas,1), float)
    random.seed(0)
    w1 = random.rand(neuronas, num_inputs)
    w2 = random.rand(output, neuronas)
    
    neurons_red = PerceptronMulti(x_i, d, w1, w2, umbral, umbral_ocultas, prec, epoch, f_ac_ap, neuronas, num_inputs, output)
    epochs,w1_a,w2_a,us_a,uoc_a,E = neurons_red.Learn()
    
    red = PerceptronMulti(x_j,d,w1_a,w2_a,us_a,uoc_a,prec,epoch,f_ac_ap,neuronas,num_inputs,output)
    outputs = red.Operation()
    
    print("Salida esperada:", matriz_data[:, 5])
    print("Salidas de red: ", outputs)

Salida esperada: [1 1 1 1]
Salidas de red:  [[-0.00028929252480248453], [0.9897948817167282], [-9.04387234782573e-05], [0.9890422290146771]]
